[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/spatial4.ipynb)


# Interactive visualization using Vitessce

In [ ]:
!lamin connect laminlabs/lamindata

In [ ]:
import vitessce as vit
from vitessce import data_utils as vitdu
import lamindb as ln
import json

ln.track("O3BqO4LsJmxI0005")

### Create a VitessceConfig object for Xenium 

In [ ]:
xenium_aligned_1_sdata = ln.Artifact.get(key="xenium_aligned_1_guide_min.zarr").load()
xenium_aligned_1_sdata

In [ ]:
adata = xenium_aligned_1_sdata["table"]
h5ad_processed_filepath = "./xenium_aligned_1_guide_mi_table.h5ad"
adata.write_h5ad(h5ad_processed_filepath)

In [ ]:
ref_spec = vitdu.generate_h5ad_ref_spec(h5ad_processed_filepath)

In [ ]:
ref_path = "./xenium_aligned_1_guide_min.reference.json"

with open(ref_path, "w") as file:
    json.dump(ref_spec, file)

In [ ]:
ref_artifact = ln.Artifact(
    ref_path,
    key="vitessce_examples/xenium_aligned_1_guide_min.reference.json",
    description="Reference JSON for xenium_aligned_1_guide_min",
).save()

In [ ]:
ref_artifact._branch_code = 0

In [ ]:
vc = vit.VitessceConfig(
    schema_version="1.0.17",
    description="Xenium Aligned 1 Guide Min",
)

In [ ]:
dataset_uid = "sdata_xenium"

dataset = vc.add_dataset(name="Xenium Aligned 1 Guide Min").add_object(
    vit.SpatialDataWrapper(
        sdata_artifact=ln.Artifact.get(key="xenium_aligned_1_guide_min.zarr"),
        ref_spec=ref_spec,
        image_path="images/morphology_focus",
        table_path="tables/table",
        obs_feature_matrix_path="tables/table/X",
        obs_spots_path="shapes/cell_circles",
        coordinate_system="global",
    )
)

In [ ]:
# Create views
spatial = vc.add_view(vit.ViewType.SPATIAL, dataset=dataset)
feature_list = vc.add_view(vit.ViewType.FEATURE_LIST, dataset=dataset)
layer_controller = vc.add_view(vit.ViewType.LAYER_CONTROLLER, dataset=dataset)

# Initialize visual properties for multiple linked views:
vc.link_views_by_dict(
    [spatial, layer_controller],
    {
        "imageLayer": vit.CoordinationLevel(
            [
                {
                    "photometricInterpretation": "RGB",
                }
            ]
        ),
    },
    scope_prefix=vit.get_initial_coordination_scope_prefix(dataset_uid, "image"),
)

vc.link_views([spatial, layer_controller, feature_list], ["obsHighlight"])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller))

In [ ]:
description_xenium_aligned_1_sdata = "View xenium_aligned_1_sdata in Vitessce"

h5ad_vc_artifact = ln.integrations.save_vitessce_config(
    vc,
    description=description_xenium_aligned_1_sdata,
)

In [ ]:
ln.finish()